In [26]:
import pandas as pd
import numpy as np
from path import Path
import torch.utils.data as data
from imageio import imread
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F 
import matplotlib.pyplot as plt
%matplotlib inline 
from torch.utils.data.sampler import SubsetRandomSampler

from PIL import *
import ast

In [27]:
import numpy as np

class ScaledDotProductAttention(nn.Module):
    ''' Scaled Dot-Product Attention '''
    # given query, key,value it finds the rightful weighted component of v to get the attention applied ouput
    #q,v,k- batch X length of sequence X features or encoding
    #attention sholuld be -batchX7X7
    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, q, k, v, mask=None):
#         print(k.transpose(1,2).shape)

        attn = torch.bmm(q, k.transpose(1, 2)) 
#         print(attn.shape)
        attn = attn / self.temperature

        if mask is not None:
            attn = attn.masked_fill(mask, -np.inf)

        attn = self.softmax(attn)
        attn = self.dropout(attn)
#         print(str(attn.shape)+" "+str(v.shape))
        output = torch.bmm(attn, v)

        return output, attn
SDP=ScaledDotProductAttention(5)
Ss=SDP(torch.zeros(5,6,100),torch.zeros(5,6,100),torch.zeros(5,6,100))
# print(Ss[0].shape)
class MultiHeadAttention(nn.Module):
    ''' Multi-Head Attention module '''

    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        self.w_qs = nn.Linear(d_model, n_head * d_k)
        self.w_ks = nn.Linear(d_model, n_head * d_k)
        self.w_vs = nn.Linear(d_model, n_head * d_v)
        nn.init.normal_(self.w_qs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_ks.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_vs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_v)))

        self.attention = ScaledDotProductAttention(temperature=np.power(d_k, 0.5))
        self.layer_norm = nn.LayerNorm(d_model)

        self.fc = nn.Linear(n_head * d_v, d_model)
        nn.init.xavier_normal_(self.fc.weight)

        self.dropout = nn.Dropout(dropout)


    def forward(self, q, k, v, mask=None):

        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head

        sz_b, len_q, _ = q.size()
        sz_b, len_k, _ = k.size()
        sz_b, len_v, _ = v.size()
#         print(str(sz_b)+"die")

        residual = q

        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)

        q = q.permute(2, 0, 1, 3).contiguous().view(-1, len_q, d_k) # (n*b) x lq x dk
        k = k.permute(2, 0, 1, 3).contiguous().view(-1, len_k, d_k) # (n*b) x lk x dk
        v = v.permute(2, 0, 1, 3).contiguous().view(-1, len_v, d_v) # (n*b) x lv x dv
#         print("v-"+str(v.shape))
        if mask is not None:
            mask = mask.repeat(n_head, 1, 1) # (n*b) x .. x ..
        output, attn = self.attention(q, k, v, mask=mask)
#         print(q.shape,k.shape,v.shape)
        output = output.view(n_head, sz_b, len_q, d_v)
        output = output.permute(1, 2, 0, 3).contiguous().view(sz_b, len_q, -1) # b x lq x (n*dv)

        output = self.dropout(self.fc(output))
        output = self.layer_norm(output + residual)

        return output, attn
MHA=MultiHeadAttention(4,15,15,15)
op=MHA(torch.zeros(5,7,15),torch.zeros(5,7,15),torch.zeros(5,7,15))
# print(op[0].shape)
class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Conv1d(d_in, d_hid, 1) # position-wise
        self.w_2 = nn.Conv1d(d_hid, d_in, 1) # position-wise
        self.layer_norm = nn.LayerNorm(d_in)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        residual = x
        output = x.transpose(1, 2)
#         print("FCC-"+str(output.shape))
#         print("FFC_out-"+str(self.w_1(output).shape))
        output = self.w_2(F.relu(self.w_1(output)))
        output = output.transpose(1, 2)
        output = self.dropout(output)
        output = self.layer_norm(output + residual)
        return output

    

In [28]:
class EncoderLayer(nn.Module):
    ''' Compose with two layers '''

    def __init__(self, d_model, d_inner, n_head, d_k, d_v, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.slf_attn = MultiHeadAttention(
            n_head, d_model, d_k, d_v, dropout=dropout)
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner, dropout=dropout)

    def forward(self, enc_input, non_pad_mask=None, slf_attn_mask=None):
        enc_output, enc_slf_attn = self.slf_attn(
            enc_input, enc_input, enc_input, mask=slf_attn_mask)
#         print("ENC_o")
#         print(enc_output.shape)


        enc_output = self.pos_ffn(enc_output)


        return enc_output, enc_slf_attn
    
XX=EncoderLayer(15,10,4,10,10)

zz=XX(torch.zeros(5,7,15))
# print("ENc")
# print(zz[0].shape)
# print("start")
class Encoder(nn.Module):
    ''' A encoder model with self attention mechanism. '''

    def __init__(self,n_modality,d_model,n_head,d_k,d_v,dropout,n_layers,d_inner=500):
        #d_model - number of features in input 100 here
        #n_head - number of heads of multihaded attention
        #d_k=d_q=  number of features in query, key
        #d_v = number of features in value whose weighted(attentioned) sum we gonna take
        

        super().__init__()
        self.n_modality=n_modality
#         self.stn=nn.ModuleList([SpatialTransformer(3, (240,240), 8) for _ in range(n_ref)])
        
        self.layer_stack = nn.ModuleList([EncoderLayer(d_model, d_inner, n_head, d_k, d_v, dropout=dropout) 
                                          for _ in range(n_layers)])
        self.em=nn.Linear(48,100)
        self.em1=nn.Linear(225,100)
        self.fc1=nn.Linear(d_model*n_modality,300)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(300,100)
        self.fc3=nn.Linear(100,3)
#         self.fc4=nn.Linear(50,3)
        self.bn1 = nn.BatchNorm1d(num_features=300)
        self.bn2 = nn.BatchNorm1d(num_features=100)
        self.softmax=nn.Softmax(1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, embeddings1,embeddings2,embeddings3 ):#mlpxlstmx3d


        
        encodings_total=[self.em1(embeddings1),embeddings2,self.em(embeddings3)]

        enc_output=torch.stack(encodings_total,0)

        
        enc_output=enc_output.permute(1,0,2)
#         print("encoding_OUTPUT2-"+str(enc_output.shape))

        for enc_layer in self.layer_stack:
            enc_output, enc_slf_attn = enc_layer(enc_output,non_pad_mask=None,slf_attn_mask=None)
           
        
        final_input=enc_output.reshape(enc_output.shape[0],-1)

        final=self.relu(self.fc3(self.bn2(self.relu((self.fc2(self.bn1(self.relu(self.fc1(final_input)))))))))
        
        return(final)
     

In [29]:
e1=torch.zeros(4,100)
e2=torch.zeros(4,100)
e3=torch.stack([e1,e2],0)
e3.shape

torch.Size([2, 4, 100])

In [30]:
feature_set_MLP=pd.read_csv('ILD_MLP_features_ankit.csv')
feature_set_3d=pd.read_csv('AnkitILD_3d.csv')
feature_set_LSTM=pd.read_csv('ILD_LSTM_features_ankit.csv')
feature_set_3d.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_38,tag_39,tag_40,tag_41,tag_42,tag_43,tag_44,tag_45,tag_46,tag_47
0,hspa6,-8.484847e-07,-3.584066e-08,2.383717e-01,3.316614e-07,-6.006747e-08,-2.536388e-01,5.777697e-07,-2.904141e-08,-2.503161e-01,...,-2.534240e-01,-1.654295e-06,-1.755535e-07,-2.540403e-01,0.000001,8.596015e-08,-2.551997e-01,1.668114e-06,-7.926568e-08,2.475293e-01
1,mapk1,-3.248563e-06,-2.458299e-01,-9.050885e-07,1.842945e-06,2.537970e-01,-6.018031e-07,1.580375e-06,2.480742e-01,3.703919e-07,...,1.010100e-06,-6.833384e-07,2.566224e-01,8.035854e-07,0.000002,-2.483860e-01,3.781214e-07,-3.394537e-07,2.516636e-01,8.085547e-07
2,cnot7,-2.439943e-01,-1.255201e-03,-8.056722e-05,2.425974e-01,1.127687e-03,3.797804e-05,2.433885e-01,1.089189e-03,2.893518e-05,...,3.821624e-05,-2.444437e-01,1.537090e-03,3.304550e-05,0.237218,-1.033907e-03,1.390690e-05,2.301513e-01,1.252605e-03,-6.323292e-06
3,tirap,-1.524291e-01,-5.215052e-03,-1.798938e-04,1.514273e-01,4.854097e-03,9.117917e-05,1.519139e-01,4.738462e-03,1.772933e-04,...,1.944128e-04,-1.550333e-01,5.170598e-03,1.695126e-04,0.146611,-4.673900e-03,1.482207e-04,1.476918e-01,4.311043e-03,-5.247627e-05
4,scin,3.260720e-06,-2.457124e-01,4.632095e-08,-9.903713e-07,2.543698e-01,8.670664e-07,-2.894644e-06,2.481833e-01,1.280780e-07,...,6.828643e-07,2.958956e-06,2.566543e-01,2.668122e-07,-0.000003,-2.480078e-01,4.687189e-07,-1.864615e-06,2.511671e-01,-6.646636e-08


In [31]:
feature_set_MLP.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_215,tag_216,tag_217,tag_218,tag_219,tag_220,tag_221,tag_222,tag_223,tag_224
0,hspa6,0.0,3.500152,0.0,4.197427,0.0,0.0,2.504990,4.630731,3.426645,...,0.0,3.367921,2.667888,0.0,0.0,0.0,0.0,0.0,0.702599,3.955442
1,scarb1,0.0,3.895088,0.0,4.644502,0.0,0.0,2.676440,5.127488,4.194887,...,0.0,3.544901,2.634494,0.0,0.0,0.0,0.0,0.0,1.263837,4.390254
2,mapk1,0.0,3.197021,0.0,3.810046,0.0,0.0,2.164615,3.919276,3.232924,...,0.0,2.883429,2.289313,0.0,0.0,0.0,0.0,0.0,0.687884,3.244735
3,adam32,0.0,1.336279,0.0,1.620950,0.0,0.0,0.971384,1.830700,1.393075,...,0.0,1.329008,0.997157,0.0,0.0,0.0,0.0,0.0,0.352354,1.665234
4,spata17,0.0,1.768997,0.0,1.791337,0.0,0.0,0.706958,2.092716,1.662794,...,0.0,2.180046,1.217243,0.0,0.0,0.0,0.0,0.0,0.788542,1.739285


In [32]:
header_of_MLP=['tag_'+str(i) for i in range(feature_set_MLP.shape[1]-1)]
features_MLP=np.array(feature_set_MLP[header_of_MLP])
gene_MLP=feature_set_MLP['gene_name']
print(features_MLP.shape)
print(len(gene_MLP))

(18144, 225)
18144


In [33]:
dictionary_MLP={}
u=0
for gn in gene_MLP:
    dictionary_MLP[gn]=features_MLP[u]
    u=u+1

In [34]:
header_of_3d=['tag_'+str(i) for i in range(feature_set_3d.shape[1]-1)]
features_3d=np.array(feature_set_3d[header_of_3d])
gene_3d=feature_set_3d['gene_name']
print(features_3d.shape)
print(len(gene_3d))


(5166, 48)
5166


In [35]:
dictionary_3d={}
u=f=0
for gn in gene_3d:
    if gn in dictionary_3d.keys():
#         print(gn)
        f=f+1
    dictionary_3d[gn]=features_3d[u]
    u=u+1
print(u)

5166


In [36]:
print(len(dictionary_3d.keys()))

3330


In [37]:
header_of_lstm=['tag_'+str(i) for i in range(feature_set_LSTM.shape[1]-1)]
features_lstm=np.array(feature_set_LSTM[header_of_lstm])
gene_lstm=feature_set_LSTM['gene_name']
print(features_lstm.shape)
print(len(gene_lstm))

(13716, 100)
13716


In [38]:
dictionary_lstm={}
u=f=0
for gn in gene_lstm:
    if gn in dictionary_lstm.keys():
#         print(gn)
        f=f+1
    dictionary_lstm[gn]=features_lstm[u]
    u=u+1
print(u)

13716


In [39]:
fil=open('../../Multi-modality/Model/ILD/data/labels_ILD.txt','r')
tmp=list()
for line in fil:
	tmp.append(int(line))

label_ILD=np.array(tmp)
print(label_ILD.shape)

(18144,)


In [40]:
class Sequenceloader(data.Dataset):
    def __init__(self,GN,label,Feat):
        self.gene_names=GN
        self.features_mlp=Feat
        self.label=label
        self.coincdgene_name=[]
        self.coincidfeature_lstm=[]
        self.coincidfeature_3d=[]
        self.coincidfeature_mlp=[]
        self.coincidlabel=[]
        for i in range(len(self.gene_names)):
            u=self.gene_names[i]
            if u in dictionary_3d.keys() and u in dictionary_lstm.keys():
                
                if np.array(self.label[i])==2:
                    ch=3
                else:
                    ch=1
                    
                for jj in range(ch):
                    self.coincdgene_name.append(u)
                    self.coincidfeature_lstm.append(dictionary_lstm[u])
                    self.coincidfeature_3d.append(dictionary_3d[u])
                    self.coincidlabel.append(self.label[i])
                    self.coincidfeature_mlp.append(self.features_mlp[i])
        
        
    
    def __len__(self):
#         print(len(self.dataset))
        return len(self.coincdgene_name)       

    def __getitem__(self, index):
         return np.array(self.coincidfeature_mlp[index]),np.array(self.coincidfeature_lstm[index]),np.array(self.coincidfeature_3d[index]),np.array(self.coincidlabel[index])
#         print(self.dataset['gen_name'][index])          
#         try:
           
#         except :
            
total_set=Sequenceloader(gene_MLP,label_ILD,features_MLP)  
a=b=c=0
for x1,x2,x3,z in total_set:
    if(z==0):
        a=a+1
    elif z==1:
        b=b+1
    else:
        c=c+1
print(a,b,c)
    

1963 2363 2520


In [41]:
batch_size = 4
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(total_set)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(total_set, batch_size=batch_size, 
                                           sampler=train_sampler,drop_last=True)
validation_loader = torch.utils.data.DataLoader(total_set, batch_size=batch_size,
                                                sampler=valid_sampler,drop_last=True)


In [42]:
for a,b,c,d in train_loader:
    print(a.shape,b.shape,c.shape,d.shape)

torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) to

torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) to

torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) to

torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) torch.Size([4])
torch.Size([4, 225]) torch.Size([4, 100]) torch.Size([4, 48]) to

In [43]:
def test(test_loader,model):

    total_imgs=0;
    total_corrects=0
    u=0
    nb_classes=3
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    for i1,i2,i3,label in test_loader:
                

        output=model(i1.to(device).float(),i2.to(device).float(),i3.to(device).float())
        total_imgs=total_imgs+label.shape[0]
        z=torch.max(output,1)[1]==label.to(device)
        _, preds = torch.max(output, 1)
#         print(output.shape)
 
        num_corrects=torch.sum(z)
        total_corrects=total_corrects+num_corrects
        for t, p in zip(label.view(-1), preds.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1


        u=u+1
    
    
    print(confusion_matrix)
    return(total_corrects,total_imgs)

In [44]:
device='cuda:7'
model_3dXLSTMXMLP=Encoder(3,100,4,300,300,True,4).to(device)
model_3dXLSTMXMLP.load_state_dict(torch.load(Path('2AnkitILD COMBO_3DXLSTMXMLP.pt')))
uz=vz=torch.rand(4,100).to(device)
# model_3dXMLP(uz,vz).shape


In [45]:

# model.load_state_dict(torch.load(Path('currentz_m1_epoch.pt')))
optim_params = [
    {'params': model_3dXLSTMXMLP.parameters(), 'lr': 0.0001}
]
optimizer = torch.optim.Adam(optim_params)
criterion = nn.CrossEntropyLoss()

In [46]:
epoch=10000
for i in range(epoch):
    torch.save(model_3dXLSTMXMLP.state_dict(), '2AnkitILD COMBO_3DXLSTMXMLP.pt')
    print("Accuracy-"+str(test(validation_loader,model_3dXLSTMXMLP)))
    total_loss=0
    for inp1,inp2,inp3,lab in train_loader:

        

        output=model_3dXLSTMXMLP(inp1.to(device).float(),inp2.to(device).float(),inp3.to(device).float())

        loss_batch=criterion(output,lab.to(device))/4
        optimizer.zero_grad()  
        loss_batch.backward()
        optimizer.step()
        total_loss+=loss_batch
       
    print(total_loss)
    
    
        
        

tensor([[361.,  12.,  21.],
        [  1., 398.,  67.],
        [ 23.,  49., 436.]])
Accuracy-(tensor(1195, device='cuda:7'), 1368)
tensor(107.9484, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[363.,  10.,  20.],
        [  2., 410.,  54.],
        [ 25.,  54., 430.]])
Accuracy-(tensor(1203, device='cuda:7'), 1368)
tensor(112.9747, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[369.,  10.,  15.],
        [  7., 393.,  66.],
        [ 30.,  42., 436.]])
Accuracy-(tensor(1198, device='cuda:7'), 1368)
tensor(109.9219, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[370.,   9.,  14.],
        [  3., 391.,  72.],
        [ 22.,  45., 442.]])
Accuracy-(tensor(1203, device='cuda:7'), 1368)
tensor(105.5095, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[372.,   8.,  13.],
        [  4., 394.,  68.],
        [ 16.,  61., 432.]])
Accuracy-(tensor(1198, device='cuda:7'), 1368)
tensor(104.4309, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[370.,   6.,  18.],
        [  1., 404.,  

tensor(96.0464, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[377.,   7.,  10.],
        [  3., 410.,  53.],
        [ 30.,  58., 420.]])
Accuracy-(tensor(1207, device='cuda:7'), 1368)
tensor(89.0662, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[377.,   5.,  12.],
        [  4., 409.,  53.],
        [ 31.,  31., 446.]])
Accuracy-(tensor(1232, device='cuda:7'), 1368)
tensor(90.0571, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[382.,   2.,   9.],
        [  1., 403.,  62.],
        [ 23.,  49., 437.]])
Accuracy-(tensor(1222, device='cuda:7'), 1368)
tensor(94.8641, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[375.,   7.,  12.],
        [  2., 412.,  52.],
        [ 30.,  45., 433.]])
Accuracy-(tensor(1220, device='cuda:7'), 1368)
tensor(94.3066, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[370.,   7.,  16.],
        [  6., 417.,  43.],
        [ 32.,  44., 433.]])
Accuracy-(tensor(1220, device='cuda:7'), 1368)
tensor(97.2851, device='cuda:7', grad_fn=<AddBackward0>

tensor(90.8740, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[370.,   7.,  17.],
        [  1., 414.,  51.],
        [ 23.,  44., 441.]])
Accuracy-(tensor(1225, device='cuda:7'), 1368)
tensor(87.4777, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[369.,   6.,  19.],
        [  2., 434.,  29.],
        [ 15.,  72., 422.]])
Accuracy-(tensor(1225, device='cuda:7'), 1368)
tensor(91.0757, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[370.,   6.,  18.],
        [  6., 398.,  62.],
        [ 27.,  39., 442.]])
Accuracy-(tensor(1210, device='cuda:7'), 1368)
tensor(91.5608, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[380.,   3.,  11.],
        [  1., 418.,  46.],
        [ 14.,  44., 451.]])
Accuracy-(tensor(1249, device='cuda:7'), 1368)
tensor(88.7241, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[375.,   4.,  15.],
        [  3., 421.,  42.],
        [ 23.,  41., 444.]])
Accuracy-(tensor(1240, device='cuda:7'), 1368)
tensor(88.3721, device='cuda:7', grad_fn=<AddBackward0>

tensor(68.8115, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[369.,   5.,  19.],
        [  1., 436.,  29.],
        [ 19.,  38., 452.]])
Accuracy-(tensor(1257, device='cuda:7'), 1368)
tensor(70.6299, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[374.,   0.,  20.],
        [  0., 445.,  21.],
        [  8.,  57., 443.]])
Accuracy-(tensor(1262, device='cuda:7'), 1368)
tensor(69.1522, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[368.,   1.,  24.],
        [  1., 433.,  32.],
        [ 10.,  51., 448.]])
Accuracy-(tensor(1249, device='cuda:7'), 1368)
tensor(69.6068, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[374.,   1.,  19.],
        [  1., 437.,  27.],
        [ 13.,  56., 440.]])
Accuracy-(tensor(1251, device='cuda:7'), 1368)
tensor(82.1810, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[379.,   1.,  14.],
        [  1., 423.,  41.],
        [ 25.,  25., 459.]])
Accuracy-(tensor(1261, device='cuda:7'), 1368)
tensor(68.7095, device='cuda:7', grad_fn=<AddBackward0>

tensor(66.9279, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[377.,   2.,  15.],
        [  0., 428.,  38.],
        [  9.,  17., 482.]])
Accuracy-(tensor(1287, device='cuda:7'), 1368)
tensor(71.5914, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[376.,   0.,  18.],
        [  1., 415.,  50.],
        [ 15.,  13., 480.]])
Accuracy-(tensor(1271, device='cuda:7'), 1368)
tensor(71.3593, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[378.,   3.,  12.],
        [  2., 411.,  53.],
        [ 15.,  24., 470.]])
Accuracy-(tensor(1259, device='cuda:7'), 1368)
tensor(70.4526, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[369.,   0.,  25.],
        [  0., 448.,  17.],
        [ 14.,  52., 443.]])
Accuracy-(tensor(1260, device='cuda:7'), 1368)
tensor(65.0166, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[380.,   0.,  14.],
        [  0., 431.,  35.],
        [ 17.,  27., 464.]])
Accuracy-(tensor(1275, device='cuda:7'), 1368)
tensor(70.8532, device='cuda:7', grad_fn=<AddBackward0>

...

tensor(35.4089, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[382.,   0.,  11.],
        [  0., 451.,  15.],
        [  9.,   5., 495.]])
Accuracy-(tensor(1328, device='cuda:7'), 1368)
tensor(54.5580, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[373.,   0.,  21.],
        [  1., 440.,  24.],
        [ 10.,  13., 486.]])
Accuracy-(tensor(1299, device='cuda:7'), 1368)
tensor(53.3975, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[370.,   0.,  24.],
        [  0., 453.,  13.],
        [  1.,  46., 461.]])
Accuracy-(tensor(1284, device='cuda:7'), 1368)
tensor(67.5345, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[373.,   0.,  21.],
        [  1., 438.,  26.],
        [  5.,  18., 486.]])
Accuracy-(tensor(1297, device='cuda:7'), 1368)
tensor(47.4936, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[385.,   0.,   9.],
        [  0., 441.,  24.],
        [  7.,  12., 490.]])
Accuracy-(tensor(1316, device='cuda:7'), 1368)
tensor(43.6221, device='cuda:7', grad_fn=<AddBackward0>

tensor(72.5734, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[380.,   0.,  13.],
        [  0., 442.,  24.],
        [ 13.,  22., 474.]])
Accuracy-(tensor(1296, device='cuda:7'), 1368)
tensor(51.9844, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[375.,   0.,  19.],
        [  3., 432.,  30.],
        [ 12.,   1., 496.]])
Accuracy-(tensor(1303, device='cuda:7'), 1368)
tensor(46.0109, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[379.,   0.,  15.],
        [  0., 441.,  24.],
        [  9.,   4., 496.]])
Accuracy-(tensor(1316, device='cuda:7'), 1368)
tensor(44.0634, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[379.,   0.,  14.],
        [  0., 449.,  17.],
        [  7.,  17., 485.]])
Accuracy-(tensor(1313, device='cuda:7'), 1368)
tensor(51.9471, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[373.,   0.,  21.],
        [  4., 432.,  30.],
        [  7.,   7., 494.]])
Accuracy-(tensor(1299, device='cuda:7'), 1368)
tensor(42.3536, device='cuda:7', grad_fn=<AddBackward0>

tensor(65.9670, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[372.,   0.,  22.],
        [  0., 454.,  12.],
        [  4.,  15., 489.]])
Accuracy-(tensor(1315, device='cuda:7'), 1368)
tensor(35.6166, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[380.,   0.,  13.],
        [  3., 455.,   8.],
        [ 11.,   6., 492.]])
Accuracy-(tensor(1327, device='cuda:7'), 1368)
tensor(53.4203, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[385.,   0.,   9.],
        [  2., 419.,  45.],
        [ 10.,  10., 488.]])
Accuracy-(tensor(1292, device='cuda:7'), 1368)
tensor(44.6098, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[381.,   0.,  13.],
        [  0., 446.,  20.],
        [ 11.,  15., 482.]])
Accuracy-(tensor(1309, device='cuda:7'), 1368)
tensor(47.7941, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[385.,   0.,   9.],
        [  3., 434.,  29.],
        [ 13.,   8., 487.]])
Accuracy-(tensor(1306, device='cuda:7'), 1368)
tensor(40.8936, device='cuda:7', grad_fn=<AddBackward0>

tensor(36.7202, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[382.,   0.,  11.],
        [  0., 451.,  15.],
        [  7.,   8., 494.]])
Accuracy-(tensor(1327, device='cuda:7'), 1368)
tensor(56.6073, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[379.,   0.,  15.],
        [  2., 446.,  18.],
        [  8.,   6., 494.]])
Accuracy-(tensor(1319, device='cuda:7'), 1368)
tensor(48.4471, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[377.,   0.,  17.],
        [  3., 447.,  15.],
        [  1.,  15., 493.]])
Accuracy-(tensor(1317, device='cuda:7'), 1368)
tensor(46.6355, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[382.,   0.,  12.],
        [  6., 435.,  25.],
        [  3.,  11., 494.]])
Accuracy-(tensor(1311, device='cuda:7'), 1368)
tensor(42.2758, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[382.,   0.,  11.],
        [  1., 443.,  22.],
        [  6.,   5., 498.]])
Accuracy-(tensor(1323, device='cuda:7'), 1368)
tensor(38.1549, device='cuda:7', grad_fn=<AddBackward0>

KeyboardInterrupt: 

In [26]:
!nvidia-smi
tensor([[375.,   3.,  15.],
        [  1., 432.,  33.],
        [ 11.,  50., 448.]])
Accuracy-(tensor(1255, device='cuda:7'), 1368)

Thu May 30 13:48:59 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0  On |                  N/A |
| 56%   73C    P2   180W / 250W |  10666MiB / 11172MiB |     98%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 32%   56C    P2    74W / 250W |   4404MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G